In [14]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\jaesc2\GitHub\skforecast


In [15]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.model_selection._validation import _backtesting_forecaster
from skforecast.preprocessing import RollingFeatures

# Fixtures
from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.model_selection.tests.fixtures_model_selection import y
from skforecast.model_selection.tests.fixtures_model_selection import exog
from skforecast.model_selection.tests.fixtures_model_selection import out_sample_residuals

In [16]:
print(np.__version__)
print(pd.__version__)
import skforecast
print(skforecast.__version__)

1.26.4
2.2.3
0.14.0


In [20]:
expected_metric = pd.DataFrame({"mean_absolute_error": [0.2635147997223325]})
expected_predictions = pd.DataFrame(
    {
        "pred": np.array(
            [
                0.37683092, 0.27485253, 0.35459493, 0.50933738, 0.40740455,
                0.44359783, 0.47699345, 0.53865623, 0.30365329, 0.34961539,
                0.5671574 , 0.56751209
            ]
        )
    },
    index=pd.RangeIndex(start=38, stop=50, step=1),
)

window_features = RollingFeatures(
    stats = ['mean', 'min', 'max', 'sum', 'median', 'ratio_min_max'],
    window_sizes = 3,
)
forecaster = ForecasterRecursive(
    regressor=LinearRegression(), lags=3, window_features=window_features
)

n_backtest = 12
y_train = y[:-n_backtest]
cv = TimeSeriesFold(
        steps                 = 4,
        initial_train_size    = len(y_train),
        window_size           = None,
        differentiation       = None,
        refit                 = True,
        fixed_train_size      = False,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
    )
metric, backtest_predictions = _backtesting_forecaster(
                                    forecaster = forecaster,
                                    y          = y,
                                    exog       = exog,
                                    cv         = cv,
                                    metric     = 'mean_absolute_error',
                                    verbose    = False
                                )

pd.testing.assert_frame_equal(expected_metric, metric)
pd.testing.assert_frame_equal(expected_predictions, backtest_predictions)

100%|██████████| 3/3 [00:00<00:00, 124.48it/s]


In [22]:
expected_metric = pd.DataFrame({"mean_absolute_error": [0.24696432911619762]})
expected_predictions = pd.DataFrame(
    {
        "pred": np.array(
            [
                0.32372783, 0.43458024, 0.77238875, 0.60657529, 0.37897279,
                0.542262  , 0.31403908, 0.37007796, 0.35614498, 0.51622723,
                0.67746181, 0.59161821
            ]
        )
    },
    index=pd.RangeIndex(start=38, stop=50, step=1),
)

window_features = RollingFeatures(
    stats = ['mean', 'min', 'max', 'sum', 'median', 'ratio_min_max'],
    window_sizes = 3,
)
forecaster = ForecasterDirect(
    regressor=LinearRegression(), steps=4, lags=3, window_features=window_features
)

n_backtest = 12
y_train = y[:-n_backtest]
cv = TimeSeriesFold(
        steps                 = 4,
        initial_train_size    = len(y_train),
        window_size           = None,
        differentiation       = None,
        refit                 = True,
        fixed_train_size      = False,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
    )
metric, backtest_predictions = _backtesting_forecaster(
                                    forecaster = forecaster,
                                    y          = y,
                                    exog       = exog,
                                    cv         = cv,
                                    metric     = 'mean_absolute_error',
                                    verbose    = False
                                )

pd.testing.assert_frame_equal(expected_metric, metric)
pd.testing.assert_frame_equal(expected_predictions, backtest_predictions)

100%|██████████| 3/3 [00:00<00:00, 97.30it/s]
